# Identifying and acquiring relevant X-ray observations

This tutorial will explain the basic concepts and components of the X-ray astronomy Python module 'Democratising Archival X-ray Astronomy' (DAXA). We will particularly focus on the various 'mission' classes (implemented for each of the X-ray telescopes that DAXA supports), and the functionality that allows for large numbers of observations to be selected and downloaded; we will also discuss the 'Archive' class, which can be used to create, reduce, and organise multi-mission collections of X-ray astronomy data.

## Import Statements

In [71]:
from daxa.mission import MISS_INDEX, XMMPointed, Chandra, eRASS1DE, eROSITACalPV, NuSTARPointed, Swift, \
    ROSATAllSky, ROSATPointed, ASCA, INTEGRALPointed, Suzaku
from daxa.mission.base import SRC_TYPE_TAXONOMY

from datetime import date
import numpy as np

## Missions and Archives

## What missions are available?

We have implemented support for access and searching the archives of many X-ray telescopes; we would also be willing to provide an interface to the data archives of other X-ray telescopes, if feasible - please feel free to reach out using the support page if you think there is one we should add.

Some telescopes (such as the ROSATPointed/ROSATAllSky and eROSITACalPV/eRASS1DE classes) are not uniquely represented by a single DAXA mission - this is generally the case when the telescope in question has been used in distinctly different ways (e.g. ROSAT had a survey phase and a pointed phase), such that the data for one mode may not be relevant to all applications.

In [5]:
MISS_INDEX

{'xmm_pointed': daxa.mission.xmm.XMMPointed,
 'nustar_pointed': daxa.mission.nustar.NuSTARPointed,
 'erosita_calpv': daxa.mission.erosita.eROSITACalPV,
 'erosita_all_sky_de_dr1': daxa.mission.erosita.eRASS1DE,
 'chandra': daxa.mission.chandra.Chandra,
 'rosat_all_sky': daxa.mission.rosat.ROSATAllSky,
 'rosat_pointed': daxa.mission.rosat.ROSATPointed,
 'swift': daxa.mission.swift.Swift,
 'suzaku': daxa.mission.suzaku.Suzaku,
 'asca': daxa.mission.asca.ASCA,
 'integral_pointed': daxa.mission.integral.INTEGRALPointed}

### Notes on specific mission classes

Here we provide any extra notes that we think might be necessary for each mission class.

#### XMM-Pointed

This class is for acquiring XMM-Newton data, particularly that taken when the telescope is pointing at specific targets; i.e. it cannot be used to filter and download data taken when the telescope is slewing from one target to the next. 

The three EPIC instruments are supported (PN, MOS1, and MOS2), as well as the two grating spectrometers (RGS1 and RGS2). We also support the acquisition of XMM optical monitor (OM) data, but support for processing it is more limited.

In [ ]:
xm = XMMPointed()

#### Chandra

In [26]:
ch = Chandra()

#### eROSITA All-Sky DR1 (German Half)

In [14]:
ea = eRASS1DE()

#### eROSITA Calibration and Performance Verification

In [15]:
ecpv = eROSITACalPV()

#### NuSTAR-Pointed

In [18]:
nu = NuSTARPointed()

#### Swift

In [37]:
sw = Swift()

/Users/dt237/code/DAXA/daxa/mission/swift.py:97: UserWarning: 538 of the 350053 observations located for Swift have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/Users/dt237/code/DAXA/daxa/mission/swift.py:97: UserWarning: 17 of the 350053 observations located for Swift have been removed due to all chosen instrument (XRT, BAT) exposures being zero.
  self._fetch_obs_info()


#### ROSAT All-Sky Survey

In [19]:
ra = ROSATAllSky()

#### ROSAT-Pointed

In [20]:
rp = ROSATPointed()

#### Suzaku

In [21]:
su = Suzaku()

/Users/dt237/code/DAXA/daxa/mission/suzaku.py:92: UserWarning: 14 of the 3055 observations located for Suzaku have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


#### ASCA

In [22]:
asca = ASCA()

/Users/dt237/code/DAXA/daxa/mission/asca.py:87: UserWarning: 5 of the 3079 observations located for ASCA have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


#### INTEGRAL-Pointed

In [67]:
inte = INTEGRALPointed()

/Users/dt237/code/DAXA/daxa/mission/integral.py:106: UserWarning: 241 of the 203747 observations located for INTEGRAL have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/Users/dt237/code/DAXA/daxa/mission/integral.py:106: UserWarning: 7241 of the 203747 observations located for INTEGRAL have been removed due to all chosen instrument (JEMX1, JEMX2, ISGRI) exposures being zero.
  self._fetch_obs_info()


## Mission properties

Here we run through the basic properties that each of the DAXA mission classes share. We also show examples, particularly in cases where differences between telescopes result in us assigning different values for particular properties.

### Name

The name assigned to each mission class, so that they can be differentiated both by the user and by DAXA functions. Each mission class has two names, the 'internal DAXA name' (used by DAXA to identify missions) and the 'pretty name', which is typically in a more aethsetically pleasing format.

For instance, we show the 'name' and 'pretty name' of the XMMPointed and eROSITA All-Sky Survey 1 classes:

In [23]:
print(xm.name)
print(xm.pretty_name, '\n')

print(er.name)
print(er.pretty_name)

xmm_pointed
XMM-Newton Pointed 

erosita_all_sky_de_dr1
eRASS DE:1


### All Instruments & Chosen Instruments

Most telescopes have multiple instruments

In [27]:
print(rp.all_mission_instruments)
print(ch.all_mission_instruments)

['PSPCB', 'PSPCC', 'HRI']
['ACIS-I', 'ACIS-S', 'HRC-I', 'HRC-S', 'HETG', 'LETG']


### ObsID Regular Expression

Each of the mission's observations are uniquely identified by an 'ObsID', and each telescope/mission has a different format of ObsID (generally just made up of numeric characters) - there are points where the mission class may have to check the format of a supplied ObsID, and it does that by comparing to the ObsID regular expression:

In [32]:
print(xm.id_regex)
print(rp.id_regex)

^[0-9]{10}$
^(RH|rh|RP|rp|RF|rf|WH|wh|WP|wp|WF|wf)\d{6}([A-Z]\d{2}|)$


### Field of View

The field of view (FoV) values attached to DAXA mission classes represent the half-width (or radius) of the region of sky that a telescope/instrument observes. Given that each telescope instrument tends to have a unique geometry, this is a simplification, but it is beneficial to have a single number that represents how much of the sky an instrument can see.

In the simplest cases, the FoV property is just a single quantity, meaning that there is either only one instrument, or that every instrument has the same field of view. In other cases there may be multiple instruments associated with a mission, in which case they will all have their own entry in a FoV dictionary.

Finally, some telescopes (such as Chandra) have instruments which have irregular geometries (the ACIS-S and HRC-S chips), or that are frequently used in different observational modes that enable and disable different parts of the sensor. As such the actual coverage of the FoV can vary dramatically, in cases like these we will have gone with the half-width of the longest possible side of the field of view.

In [46]:
print(xm.pretty_name, '-', xm.fov, '\n')
print(rp.pretty_name, '-', rp.fov, '\n')
print(sw.pretty_name, '-', sw.fov, '\n')
print(ch.pretty_name, '-', ch.fov)

XMM-Newton Pointed - 15.0 arcmin 

ROSAT Pointed - {'PSPCB': <Quantity 60. arcmin>, 'PSPCC': <Quantity 60. arcmin>, 'HRI': <Quantity 19. arcmin>} 

Swift - {'XRT': <Quantity 11.8 arcmin>, 'BAT': <Quantity 50. arcmin>, 'UVOT': <Quantity 8.5 arcmin>} 

Chandra - {'ACIS-I': <Quantity 27.8 arcmin>, 'ACIS-S': <Quantity 27.8 arcmin>, 'HRC-I': <Quantity 49.5 arcmin>, 'HRC-S': <Quantity 49.5 arcmin>}


/var/folders/td/gw9qkx6d3szb1nkt_cfvcbzm000vzl/T/ipykernel_48957/579883568.py:4: UserWarning: Chandra FoV are difficult to define, as they can be strongly dependant on observation mode; as such take these as very approximate.
  print(ch.pretty_name, '-', ch.fov)


### Coordinate Frame

This property contains the coordinate frame for the central positions of observations taken by the mission - this is largely irrelevant to the user, and will be used in cases where the mission class needs to compare an input coordinate to an observation coordinate. 

Also, practically speaking the difference between the ICRS and FK5 frames (most commonly used) is neglible compared to the typical spatial uncertainty involved in X-ray data:

In [36]:
print(ch.coord_frame)
print(asca.coord_frame)

<class 'astropy.coordinates.builtin_frames.icrs.ICRS'>
<class 'astropy.coordinates.builtin_frames.fk5.FK5'>


### Observation Information

One of the most important properties of a DAXA mission class - this returns a dataframe of all the observations that are associated with a mission. This can include observations that are not yet publicly available (i.e. they are still in a proprietary period), but will never include observations that are planned but haven't been taken yet. 

In most cases this data is dynamically updated when a mission is declared (i.e. the table is pulled down from a mission server) - this is not the case for eROSITACalPV and eRASS1DE. **Please also note that the Swift and INTEGRALPointed missions have very large `all_obs_info` tables due to the way their data/observations are organised.**

Some information will be constant across telescopes, and some will be mission specific. We present truncated versions of `all_obs_info` for every DAXA mission as of the current date:

In [57]:
date.today().strftime('%d-%B-%Y')

'02-February-2024'

In [38]:
xm.all_obs_info

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
0,64.925415,55.999440,0000110101,2001-08-19 07:05:23,True,0 days 09:08:33,2002-09-29,310,True,2001-08-19 16:13:56
1,263.674950,-32.581670,0001730201,2001-03-09 12:44:21,True,0 days 04:44:43,2002-05-25,229,True,2001-03-09 17:29:04
2,263.674950,-32.581670,0001730301,2001-03-09 17:30:16,True,0 days 02:36:02,2002-05-25,229,True,2001-03-09 20:06:18
3,263.674950,-32.581670,0001730401,2001-03-09 09:41:25,True,0 days 03:00:59,2002-05-25,229,True,2001-03-09 12:42:24
4,99.349995,6.135278,0001730501,2002-09-17 18:35:28,True,0 days 06:05:39,2004-12-31,508,True,2002-09-18 00:41:07
...,...,...,...,...,...,...,...,...,...,...
17412,65.047250,-44.889944,0920220601,2024-01-28 04:31:51,True,0 days 10:16:40,NaT,4420,False,2024-01-28 14:48:31
17413,39.863750,-8.123889,0922880701,2024-01-27 06:06:28,True,0 days 03:36:40,NaT,4420,False,2024-01-27 09:43:08
17414,77.827208,-38.651694,0923561301,2024-01-27 10:38:49,True,0 days 04:26:40,NaT,4420,False,2024-01-27 15:05:29
17415,34.696000,0.120889,0923640101,2024-01-27 16:08:34,True,0 days 11:23:20,NaT,4420,False,2024-01-28 03:31:54


In [39]:
ch.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
0,274.43140,-33.01883,6616,True,True,2006-02-24 04:33:41.000003,2007-09-21 00:26:11.000003,573 days 19:52:30,2007-02-28,GS,ACIS-S,HETG,CC_000A8
1,83.63292,22.01447,7587,True,True,2007-02-03 09:58:57.000000,2008-07-21 21:53:57.000000,534 days 11:55:00,2008-02-06,SNR,ACIS-S,HETG,TE_0077C
2,202.50000,47.20000,13814,True,True,2012-09-20 07:21:41.999999,2012-09-22 12:47:41.999999,2 days 05:26:00,2013-10-11,NGS,ACIS-S,NONE,TE_00958
3,266.41667,-29.00781,13842,True,True,2012-07-21 11:52:41.000002,2012-07-23 17:08:41.000002,2 days 05:16:00,2012-07-25,NGS,ACIS-S,HETG,TE_008D0
4,316.72458,38.74942,13651,True,True,2012-02-13 20:18:26.999997,2012-02-16 00:51:26.999997,2 days 04:33:00,2013-02-21,MISC,HRC-S,LETG,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22946,54.19708,0.58997,1153,True,True,1999-09-05 04:29:00.999998,1999-09-05 04:29:00.999998,0 days 00:00:00,2002-10-04,CAL,HRC-S,NONE,SCENTER
22947,332.17000,45.74231,24645,True,True,2020-10-25 12:05:32.000001,2020-10-25 12:05:32.000001,0 days 00:00:00,2020-10-27,CAL,HRC-S,NONE,SCENTER
22948,350.85750,58.81483,1409,True,True,1999-10-23 18:29:33.999999,1999-10-23 18:29:33.999999,0 days 00:00:00,1999-12-09,SNR,HRC-I,NONE,
22949,84.91458,-69.74361,62437,True,True,1999-09-05 00:50:10.999997,1999-09-05 00:50:10.999997,0 days 00:00:00,2002-03-27,GS,HRC-S,NONE,SCENTER


In [58]:
ea.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,ra_min,ra_max,dec_min,dec_max,neigh_obs
0,116.703297,42.008289,117048,True,2019-12-12,2020-06-10,181 days,114.725275,118.681319,40.5,43.5,"114045,118045,122045,113048,121048,112051,1160..."
1,120.659341,42.008289,121048,True,2019-12-12,2020-06-10,181 days,118.681319,122.637363,40.5,43.5,"118045,122045,117048,125048,116051,119051,1230..."
2,124.615385,42.008289,125048,True,2019-12-12,2020-06-10,181 days,122.637363,126.593407,40.5,43.5,"122045,126045,121048,129048,123051,127051,0000..."
3,128.571429,42.008289,129048,True,2019-12-12,2020-06-10,181 days,126.593407,130.549451,40.5,43.5,"126045,130045,125048,133048,127051,131051,0000..."
4,132.527473,42.008289,133048,True,2019-12-12,2020-06-10,181 days,130.549451,134.505495,40.5,43.5,"130045,134045,129048,136048,131051,135051,0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2442,220.000000,-87.045181,220177,True,2019-12-12,2020-06-10,181 days,200.000000,240.000000,-88.5,-85.5,"001180,191174,214174,236174,259174,180177,2601..."
2443,260.000000,-87.045181,260177,True,2019-12-12,2020-06-10,181 days,240.000000,280.000000,-88.5,-85.5,"001180,236174,259174,281174,304174,220177,3001..."
2444,300.000000,-87.045181,300177,True,2019-12-12,2020-06-10,181 days,280.000000,320.000000,-88.5,-85.5,"001180,259174,281174,304174,326174,260177,3401..."
2445,340.000000,-87.045181,340177,True,2019-12-12,2020-06-10,181 days,320.000000,360.000000,-88.5,-85.5,"001180,304174,326174,349174,011174,300177,0201..."


In [59]:
ecpv.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,Field_Name,Field_Type
0,129.550000,1.500000,300007,True,2019-11-03 02:42:50,2019-11-04 03:36:37,89627.0,EFEDS,SURVEY
1,133.860000,1.500000,300008,True,2019-11-04 03:49:16,2019-11-05 05:16:39,91643.0,EFEDS,SURVEY
2,138.140000,1.500000,300009,True,2019-11-05 05:29:18,2019-11-06 06:40:06,90648.0,EFEDS,SURVEY
3,142.450000,1.500000,300010,True,2019-11-06 07:24:46,2019-11-07 08:20:08,89722.0,EFEDS,SURVEY
4,130.331300,-78.963400,300004,True,2019-11-16 23:14:40,2019-11-18 18:17:12,154952.0,ETA_CHA,SURVEY
...,...,...,...,...,...,...,...,...,...
165,284.146250,-37.909167,700008,True,2019-10-24 11:11:19,2019-10-25 08:55:52,78273.0,1RXS_J185635_375433,EXTRAGALACTIC_FIELDS
166,281.540771,79.873726,900060,True,2019-09-24 15:27:06,2019-09-24 21:30:32,21806.0,3C390_3,EXTRAGALACTIC_FIELDS
167,281.500275,79.885376,900068,True,2019-09-28 15:49:51,2019-09-28 21:30:32,20441.0,3C390_3,EXTRAGALACTIC_FIELDS
168,281.489410,79.888214,900069,True,2019-09-29 15:23:24,2019-09-29 21:30:37,22033.0,3C390_3,EXTRAGALACTIC_FIELDS


In [60]:
nu.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag
0,83.8281,-69.2465,40001014013,True,True,2013-06-29 01:16:07.183998,2013-07-05 08:51:07.184004,6 days 07:35:00.000006,2015-09-17 00:00:00,SNR,5 days 11:20:42,5 days 11:09:58,5 days 20:32:48,5 days 20:33:46,2119,1
1,83.7759,-69.2677,40001014016,True,True,2014-04-22 21:06:07.183999,2014-04-29 09:31:07.183998,6 days 12:24:59.999999,2015-09-17 00:00:00,SNR,4 days 23:56:47,4 days 23:35:17,5 days 08:14:46,5 days 08:14:09,0,1
2,83.8965,-69.2477,40001014023,True,True,2014-08-01 23:46:07.183998,2014-08-08 02:41:07.184000,6 days 02:55:00.000002,2015-09-17 00:00:00,SNR,4 days 22:46:56,4 days 22:35:32,5 days 07:01:07,5 days 07:02:55,0,0
3,340.6530,29.6985,60401031004,True,True,2018-11-28 22:21:09.184000,2018-12-08 17:16:09.183997,9 days 18:54:59.999997,2019-12-25 00:00:00,AGN,4 days 17:35:58,4 days 17:01:43,5 days 02:48:25,5 days 02:49:43,0,0
4,344.4048,-36.9765,60901012002,True,False,2023-11-02 06:06:09.184003,2023-11-10 11:41:09.184004,8 days 05:35:00.000001,2024-05-20 00:00:00,AGN,4 days 11:03:16,4 days 10:06:55,4 days 19:02:15,4 days 19:02:36,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5368,0.0000,0.0000,90411103001,True,True,2019-01-12 19:50:51.184000,2019-01-12 19:52:39.184000,0 days 00:01:48,2019-02-06 00:00:00,TOO,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5369,0.0000,0.0000,90411102001,True,True,2019-01-12 19:49:03.184001,2019-01-12 19:50:51.184000,0 days 00:01:47.999999,2019-02-06 00:00:00,TOO,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5370,0.0000,0.0000,90411101001,True,True,2019-01-12 19:26:09.183998,2019-01-12 19:49:03.184001,0 days 00:22:54.000003,2019-02-06 00:00:00,TOO,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1
5371,0.0000,0.0000,90401202001,True,True,2019-01-12 17:46:09.184002,2019-01-12 19:26:09.183998,0 days 01:39:59.999996,2019-02-06 00:00:00,TOO,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0,1


In [61]:
sw.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,xrt_exposure,bat_exposure,uvot_exposure
0,328.70739,16.89364,00173780007,True,2005-12-25 00:42:01.999999,2005-12-28 00:00:23.000000,2 days 23:18:21.000001,MISC,1 days 02:44:09.014000,1 days 00:04:10,1 days 02:40:01.030000
1,39.95079,-25.21897,00131560002,True,2005-06-03 23:59:00.999998,2005-06-06 22:47:30.000002,2 days 22:48:29.000004,MISC,0 days 22:36:12.946000,0 days 22:52:03,0 days 17:46:44.297000
2,228.46115,30.87659,00164268014,True,2005-12-03 01:00:00.999996,2005-12-06 00:07:43.000000,2 days 23:07:42.000004,MISC,0 days 19:38:55.384000,0 days 19:59:15,0 days 00:00:00
3,350.64958,5.74990,00148833003,True,2005-08-06 00:26:11.999996,2005-08-09 00:50:56.000003,3 days 00:24:44.000007,MISC,0 days 19:04:07.125000,0 days 19:23:36,0 days 18:21:43.023000
4,75.27823,45.28513,00321174011,True,2008-08-31 02:25:00.999998,2008-09-03 00:34:06.999998,2 days 22:09:06,MISC,0 days 18:49:28.970000,0 days 21:08:01.650000,0 days 18:24:15.039000
...,...,...,...,...,...,...,...,...,...,...,...
349493,105.38284,4.99064,00074865031,True,2021-08-24 07:53:36.000001,2021-08-24 09:37:42.999997,0 days 01:44:06.999996,MISC,0 days 00:00:00,0 days 00:18:02,0 days 00:00:00
349494,195.76853,41.21908,00209969000,True,2006-05-14 08:05:27.999998,2006-05-14 09:04:47.000004,0 days 00:59:19.000006,MISC,0 days 00:00:00,0 days 01:12:16,0 days 00:00:00
349495,289.92581,60.75406,00356899000,True,2009-07-09 10:01:36.000002,2009-07-09 10:42:21.000001,0 days 00:40:44.999999,MISC,0 days 00:00:00,0 days 01:03:05.088000,0 days 00:00:00
349496,115.17669,-80.00497,00076048057,True,2021-12-17 22:29:34.000002,2021-12-18 00:01:10.000001,0 days 01:31:35.999999,MISC,0 days 00:00:00,0 days 00:06:35,0 days 00:00:00


In [62]:
ra.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category
0,263.57088,67.500,RS930521N00,True,1990-07-11,1991-08-13,0 days 11:19:06,ASK
1,276.42533,67.500,RS930522N00,True,1990-07-11,1991-08-13,0 days 11:18:11,ASK
2,96.42533,-67.500,RS932908N00,True,1990-07-11,1991-08-13,0 days 05:47:48,ASK
3,83.57088,-67.500,RS932907N00,True,1990-07-11,1991-08-13,0 days 05:47:39,ASK
4,267.27100,61.875,RS930625N00,True,1990-07-11,1991-08-13,0 days 03:44:35,ASK
...,...,...,...,...,...,...,...,...
1373,289.08783,-61.875,RS932827N00,True,1990-09-10,1990-10-08,0 days 00:02:44,ASK
1374,350.17942,-33.750,RS932354N00,True,1990-11-09,1990-12-02,0 days 00:02:42,ASK
1375,278.17942,-61.875,RS932826N00,True,1990-09-04,1990-09-30,0 days 00:02:42,ASK
1376,273.75000,-45.000,RS932537N00,True,1990-09-07,1990-09-23,0 days 00:02:42,ASK


In [63]:
rp.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
0,163.1800,57.4800,RH701867A01,True,1995-04-15 23:24:16.000001,1995-05-11 14:24:47.000001,2 days 13:29:32,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
1,203.6500,37.9100,RH900717N00,True,1997-06-04 16:13:00.999998,1997-07-13 22:26:45.000004,2 days 07:58:33,HRI,N,MISC,DEEP SURVEY,2,RDF 4_0
2,163.1800,57.4800,RH701867A04,True,1997-04-15 21:51:16.999998,1997-04-28 16:37:45.999998,2 days 06:33:41,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V3.
3,350.8700,58.8100,RH500444N00,True,1995-12-23 22:18:36.999999,1996-02-01 10:17:53.999998,2 days 02:07:27,HRI,N,SNR,CAS A,2,RDF 3_4
4,163.1800,57.4800,RH701867A02,True,1996-05-01 02:09:33.000002,1996-05-29 15:31:24.999997,2 days 01:05:24,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,84.2900,-80.4700,RP999998A02,False,1991-10-20 04:07:51.999998,1991-11-01 22:43:43.000003,0 days 00:00:00,PSPCB,N,EGE,Idle Point,2,RDF 3_4
11427,93.1800,-81.8300,RH150094N00,False,1990-07-29 22:13:42.999997,1990-07-29 22:56:58.999998,0 days 00:00:00,HRI,N,STR,Calibration Source,2,RFITS V4.
11428,218.1540,-44.2031,RH001034N00,False,1997-08-28 00:29:42.000000,1997-08-28 01:09:58.999997,0 days 00:00:00,HRI,N,MISC,,2,RDF 4_2
11429,258.1383,-23.3850,RH800067M01,False,1991-03-19 20:33:48.999997,1991-03-19 20:35:59.999997,0 days 00:00:00,HRI,N,GCL,OPHIUCHUS CLUSTER,2,RFITS V3.


In [64]:
su.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,xis0_expo,xis0_num_modes,xis1_expo,xis1_num_modes,xis2_expo,xis2_num_modes,xis3_expo,xis3_num_modes
0,91.1523,-86.6779,701018010,True,2006-04-13 16:24:07.999998,2006-04-14 01:52:19.000001,0 days 09:28:11.000003,EGS,0 days 05:29:41.700000,2,0 days 05:29:41.700000,2,0 days 05:29:25.700000,2,0 days 05:29:33.700000,2
1,48.9864,-85.5003,404019010,True,2009-07-16 14:26:31.000001,2009-07-18 03:30:15.999998,1 days 13:03:44.999997,GS,0 days 17:31:24.300000,2,0 days 17:31:24.300000,2,0 days 00:00:00,0,0 days 17:31:24.300000,2
2,239.2836,-79.2302,703059010,True,2008-10-13 15:35:23.999997,2008-10-15 21:37:21.000000,2 days 06:01:57.000003,EGS,0 days 22:49:57,2,0 days 22:49:43.500000,2,0 days 00:00:00,0,0 days 22:49:35.500000,2
3,265.9438,-76.3446,705013010,True,2010-04-14 00:16:05.000002,2010-04-14 16:45:11.000002,0 days 16:29:06,EGS,0 days 11:46:23.600000,2,0 days 11:46:18.300000,2,0 days 00:00:00,0,0 days 11:46:29,2
4,74.6207,-75.2810,404036010,True,2009-06-14 01:51:18.000003,2009-06-16 13:27:19.000002,2 days 11:36:00.999999,GS,1 days 05:42:37.800000,2,1 days 05:42:37.800000,2,0 days 00:00:00,0,1 days 05:42:29.800000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,247.6537,82.9256,706004010,True,2011-04-16 00:38:47.000003,2011-04-16 10:23:15.000000,0 days 09:44:27.999997,EGS,0 days 05:13:55.700000,3,0 days 05:13:42,2,0 days 00:00:00,0,0 days 05:13:55.600000,2
3037,116.7261,85.6829,804031010,True,2010-02-08 09:53:43.999999,2010-02-08 18:15:19.000002,0 days 08:21:35.000003,GCL,0 days 05:19:56,1,0 days 05:19:56,1,0 days 00:00:00,0,0 days 05:19:56,1
3038,112.8158,85.6965,804030010,True,2010-02-08 18:18:12.000001,2010-02-09 12:00:25.000004,0 days 17:42:13.000003,GCL,0 days 10:17:39.100000,2,0 days 10:17:39.100000,2,0 days 00:00:00,0,0 days 10:17:39.100000,2
3039,112.8340,85.6974,804030020,True,2010-02-10 04:18:27.999997,2010-02-11 06:00:24.000002,1 days 01:41:56.000005,GCL,0 days 14:36:01.100000,2,0 days 14:36:01.100000,2,0 days 00:00:00,0,0 days 14:36:01.100000,2


In [65]:
asca.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,sis_exposure,gis_exposure
0,288.0116,5.0542,48002000,True,2000-03-23 05:52:13.440,2000-04-04 00:00:25.920,11 days 18:08:12.480000,XRB,5 days 07:25:04,5 days 14:47:28
1,253.5540,39.8158,78002000,True,2000-03-01 12:08:12.480,2000-03-11 00:00:51.840,9 days 11:52:39.360000,AGN,4 days 00:45:04,5 days 04:32:16
2,14.3115,-22.4431,77036000,True,1999-12-03 11:50:38.400,1999-12-15 19:00:28.800,12 days 07:09:50.400000,AGN,4 days 08:14:08,5 days 00:42:40
3,203.8856,-34.3378,77003000,True,1999-07-19 00:40:19.200,1999-07-29 15:40:27.840,10 days 15:00:08.640000,AGN,4 days 02:03:12,4 days 16:33:20
4,266.4864,-28.9429,48004000,True,2000-03-11 00:08:29.760,2000-03-22 00:00:43.200,10 days 23:52:13.440000,XRB,3 days 22:37:04,4 days 07:30:24
...,...,...,...,...,...,...,...,...,...,...
3069,87.7568,-32.2321,77010010,True,1999-10-11 13:47:08.160,1999-10-11 14:19:32.160,0 days 00:32:24,AGN,0 days 00:06:40,0 days 00:06:40
3070,150.6550,-58.6295,15000090,True,1993-08-05 18:31:58.080,1993-08-05 19:12:43.200,0 days 00:40:45.120000,GS,0 days 00:06:56,0 days 00:05:52
3071,335.1555,-24.7349,66013000,True,1998-11-16 16:24:48.960,1998-11-18 17:34:39.360,2 days 01:09:50.400000,NGS,0 days 14:38:24,0 days 00:00:00
3072,333.8738,-17.8223,77002000,True,1999-11-17 04:45:41.760,1999-11-19 01:35:54.240,1 days 20:50:12.480000,AGN,0 days 16:30:24,0 days 00:00:00


In [68]:
inte.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,target_category,jemx1_exposure,jemx2_exposure,isgri_exposure,picsit_exposure,spi_exposure,scw_ver
0,294.97232,-89.65222,218500510010,True,True,2020-01-22 20:40:39.751343,2020-01-22 22:17:46.754676,0 days 01:37:07.003333,MISC,0 days 01:36:12,0 days 01:36:12,0 days 01:36:10,0 days 00:28:29,0 days 01:36:12,001
1,117.47166,-89.57144,218600510010,True,True,2020-01-25 11:40:02.881811,2020-01-25 12:38:50.883829,0 days 00:58:48.002018,MISC,0 days 00:57:57,0 days 00:57:57,0 days 00:57:55,0 days 00:57:27,0 days 00:57:57,001
2,166.19875,-89.54264,229200810010,True,True,2020-11-02 08:42:24.928042,2020-11-02 09:41:14.930022,0 days 00:58:50.001980,MISC,0 days 00:58:00,0 days 00:58:00,0 days 00:57:44,0 days 00:57:29,0 days 00:58:00,001
3,130.24533,-89.51414,209300110010,True,True,2019-05-21 07:51:13.039554,2019-05-21 08:24:33.041123,0 days 00:33:20.001569,MISC,0 days 00:32:25,0 days 00:32:25,0 days 00:32:23,0 days 00:32:04,0 days 00:32:25,001
4,125.61337,-89.47669,209300370010,True,True,2019-05-21 22:51:03.082145,2019-05-21 23:24:24.083715,0 days 00:33:21.001570,MISC,0 days 00:32:30,0 days 00:32:30,0 days 00:32:28,0 days 00:32:02,0 days 00:32:30,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196260,180.57124,84.86295,243300300010,True,True,2021-11-11 00:43:15.631850,2021-11-11 00:59:19.632566,0 days 00:16:04.000716,MISC,0 days 00:15:12,0 days 00:15:12,0 days 00:15:10,0 days 00:00:00,0 days 00:15:12,001
196261,320.96317,84.89636,247900760010,True,True,2022-03-15 00:56:09.682970,2022-03-15 03:26:12.688189,0 days 02:30:03.005219,MISC,0 days 00:00:00,0 days 00:00:00,0 days 00:12:46,0 days 00:00:00,0 days 02:28:43,001
196262,181.41162,84.92258,243300290010,True,True,2021-11-11 00:28:00.631174,2021-11-11 00:41:20.631765,0 days 00:13:20.000591,MISC,0 days 00:12:57,0 days 00:12:57,0 days 00:12:55,0 days 00:00:00,0 days 00:12:57,001
196263,112.34688,85.41183,246800220010,True,True,2022-02-12 08:52:03.591255,2022-02-12 09:01:56.591858,0 days 00:09:53.000603,MISC,0 days 00:09:24,0 days 00:09:24,0 days 00:09:22,0 days 00:00:00,0 days 00:09:24,001


### Filter Array

This is unlikely to ever be accessed directly by the user, but is what defines the observations that a mission currently deems to be accepted/selected. It is a boolean numpy array with a length equal to the number of observations in the all_obs_info dataframe, a `True` value means the observation is accepted and a `False` value means it is excluded; all observations start off as accepted.

Various filtering methods can be used to adjust the filter array and set the observations which are to be downloaded/included in a DAXA archive, depending on your particular sample and science case. 

It is also possible to manually set this filter array, as is demonstrated below:

In [69]:
# The filter array defaults to all True, so all observations are accepted
xm.filter_array

array([ True,  True,  True, ...,  True,  True,  True])

In [74]:
# Demonstrating manually setting a filter array - it must be boolean and be the same length as the
#  'all_obs_info' table, otherewise it will not be accepted
demo_filt_arr = np.full(len(xm.all_obs_info), True)
demo_filt_arr[0] = False
xm.filter_array = demo_filt_arr
xm.filter_array

array([False,  True,  True, ...,  True,  True,  True])

## Selecting relevant observations

Few users will wish to download, process, and maintain ___complete___ observation archives, preferring to just locate data that may be relevant to the sources that they are studying. This can be achieved with the use of several filtering methods which are built into all DAXA missions.

Here we will introduce the different filtering methods that are currently implemented for DAXA missions, but we will not provide detailed demonstrations of their use; that will be left to specific case studies designed to show scientists with different needs how DAXA can be used in ways that are most relevant to them.

### Filtering on position

### Filtering on name